In [59]:
#https://spacy.io/usage/linguistic-features

## Overall Approach

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
# Make into list of lists [name, words]
import webvtt

name_transcript = []
for caption in webvtt.read('Transcript.vtt'):
    if ":" in caption.text:
        name_transcript.append(caption.text.split(":"))
    else:
        name_transcript.append(["Team", caption.text]) # if speaker unknown then assign as team

In [38]:
def task_finder(transcript):
    # Split transcript into sentences
    sentences = transcript.split('.')
    all_sentences = [i for i in sentences if i] # removes empty strings

    # Go through each sentence and check for task patterns
    # Save these sentences into new array as task_sentences
    task_sentences = []
    task_assigned_to = []
    for s in all_sentences:
        processed = nlp(s)
        is_task = 0
        i = 0
        assigned_to = "Team" # Default to entire team assigned to task

        for token in processed:
            # if proper-noun/pronoun + 'will/should'
            if (token.pos_ == 'PROPN' or token.pos_ == 'PRON') and (token.text != 'It'):
                if(i+1 < len(s.split())):
                    next_word = processed[i+1].text
                    if (next_word == 'will') or (next_word == 'should'):
                        is_task = 1
                        if (token.pos == 'PROPN') or (token.text == 'I') or (token.text == 'you'):
                            assigned_to = token.text

            # if '(need, needs) to' in sentence
            if (is_task == 0) and ((token.text == 'need') or (token.text == 'needs')):
                if(i+1 < len(s.split())):
                    next_word = processed[i+1].text
                    if (next_word == 'to'):
                        is_task = 1
                        if processed[i-1].pos_ == 'PROPN':
                            assigned_to = processed[i-1].text
                        elif (processed[i-1].text == 'I') or (processed[i-1].text == 'you'):
                            assigned_to = processed[i-1].text

            i = i + 1

        if is_task == 1:
            task_sentences.append(s)
            task_assigned_to.append(assigned_to)
    return task_sentences, task_assigned_to

In [39]:
all_tasks = []
all_assignees = []
curr_speaker = ""
prev_speaker = ""

# goes through each chunk of someone speaking
for t in name_transcript:
    prev_speaker = curr_speaker
    curr_speaker = t[0]
    curr_tasks, curr_assigned_to = task_finder(t[1])
    
    
    if curr_tasks: 
        all_tasks.append(curr_tasks)
        for a in curr_assigned_to:
            if a: # not empty
                if a == 'I': # "I" --> current speaker
                    all_assignees.append(curr_speaker)
                elif a == 'you': # "You" --> previous speaker
                    all_assignees.append(prev_speaker)
                else:
                    all_assignees.append(a)

                    
print(all_tasks)
print(all_assignees)

[[' So what do we still What else do we need to what tasks, do we need to assign'], [' Someone should also probably get started on the front end'], [' So imagine the two of us are going to need to start a back end']]
['Team', 'Team', 'Team']


In [46]:
# Task Reassignment
# "That" --> previous task? like Alena will do that
all_tasks = ['Alena will paint the house', "I can do that", "She should clean"]
all_assignees = ['Alena', 'Dong', 'Team']


i = 0
for task in all_tasks:
    if ('will do that' in task) or ('can do that' in task):
        all_tasks[i-1] = all_tasks[i-1].replace(all_assignees[i-1], all_assignees[i])
        del all_tasks[i]
        del all_assignees[i-1]
    i += 1
    
print(all_tasks)
print(all_assignees)

['Dong will paint the house', 'She should clean']
['Dong', 'Team']


## Everything else is testing

In [3]:
# fails on "We should go to the market"
# Gets the core task and ignores fluff sentances (checks that part of speech head is verb)
task_list = []
for t in transcript:
    processed = nlp(t)
    curr_task = ''

    for token in processed:
        if token.head.pos_ == 'VERB' and (token.pos_ == 'VERB' or token.pos_ =='NOUN'):
            curr_task += token.text + ' '
    print(curr_task)
    if curr_task != '':
        task_list.append(curr_task)
task_list

set table 
cook dinner tomorrow 

go 


['set table ', 'cook dinner tomorrow ', 'go ']

In [39]:
transcript = ["I will set the table with the blue napkins.", "Alex should cook dinner tomorrow.", "It is a pretty day outside.", "We can go to the market"]
# need to add rest of action somehow
# looks for (proper-noun/pronoun + "will/should/can")
task_list = []
for t in transcript:
    processed = nlp(t)
    curr_task = ''
    i = 0

    for token in processed:
        if (token.pos_ == 'PROPN' or token.pos_ == 'PRON') and (token.text != 'It'):
            if(i+1 < len(t.split())):
                next_word = processed[i+1].text
                if (next_word == 'will') or (next_word == 'should') or (next_word == 'can'):
                    curr_task += token.text + ' ' + next_word
                    print(curr_task)
        i = i + 1
    
    if curr_task != '':
        task_list.append(curr_task)
task_list

I will
Alex should
We can


['I will', 'Alex should', 'We can']

In [8]:
doc = nlp("needed")
print([token.lemma_ for token in doc])

['need']


In [27]:
words = "She is cleaning the car"
processed = nlp(words)
i = 0

for token in processed:
    if (i+1) < len(words.split()):
        print(processed[i+1].text)
    i += 1

is
cleaning
the
car


In [40]:
doc = nlp("I will set the table with the blue napkins.")
print([token.pos_ for token in doc])

['PRON', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']
